# fiberassign on DECaLS (DR8)

In [ ]:
import os, sys, subprocess, fitsio
from collections import Counter

import numpy as np
from astropy.table import Table
from astropy.io import fits

import desimodel.io
import desimodel.focalplane
import desimodel.footprint
from desitarget.targetmask import desi_mask, obsconditions
import matplotlib.pyplot as plt

## Learning goals

In this notebook you will learn how to:

* Use observational data from DR8 to prepare it for fiberassign.
* Run fiberassign using the files generated above.
* Explore the outputs of fiberassign.


See https://desi.lbl.gov/trac/wiki/Computing/JupyterAtNERSC for instructions on configuring jupyter kernels with pre-installed DESI software at NERSC.  This tutorial was last tested with the 22.2 kernel on April 29 2022.

This notebook reuses code in the [main fiberassign notebook](https://github.com/desihub/tutorials/blob/master/FiberAssign.ipynb).

In [ ]:
# You can set this to a local path if you have the software installed and a copy of the data.
# workdir = os.path.join(os.environ['HOME'], 'scratch', 'desi', 'tutorials', 'fiberassign_obs')
workdir = os.path.join(os.environ['SCRATCH'], 'desi', 'test', 'fiberassign_obs')
os.makedirs(workdir, exist_ok=True)
os.chdir(workdir)
outdir = os.path.join(workdir, 'output')
os.makedirs(outdir, exist_ok=True)

Paths and filenames with the observational data

In [ ]:
# Change this if you are running locally.
# paths = {
#     "targets": "/home/kisner/scratch/desi/tutorials", 
#     "skies": "/home/kisner/scratch/desi/tutorials", 
#     "gfas": "/home/kisner/scratch/desi/tutorials",
# }

paths = {"targets": "/global/cfs/cdirs/desi/target/catalogs/dr8/0.31.1/targets/main/resolve/", 
         "skies": "/global/cfs/cdirs/desi/target/catalogs/dr8/0.32.0/skies/", 
         "gfas": "/global/cfs/cdirs/desi/target/catalogs/dr8/0.32.0/gfas/",
}

names = {"targets": "dr8-hp-10,66,69,75,87,105.fits", 
         "skies":"dr8-0.32.0.fits", 
         "gfas": "dr8-0.32.0.fits"}

Some more files needed to run fiberassign

In [ ]:
mtlfile = os.path.join(workdir, 'mtl.fits')
truthfile = os.path.join(workdir, 'truth.fits')
targetcutfile = os.path.join(workdir, 'targets.fits') 
skycutfile = os.path.join(workdir, 'sky.fits') 
targetfile = os.path.join(paths["targets"], "targets-{}".format(names["targets"]))
skyfile = os.path.join(paths["skies"], "skies-{}".format(names["skies"]))
gfafile = os.path.join(paths["gfas"], "gfas-{}".format(names["gfas"]))
tilefile = os.path.join(workdir, "tiles.fits")

In [ ]:
# tile selection
program = "dark"

tiles = desimodel.io.load_tiles()
bright = tiles['PROGRAM']=='BRIGHT'
    
small = ((tiles['RA']>32) & (tiles['RA']<37) & (tiles['DEC']<3) & (tiles['DEC']>-3))

if program=="bright":
    Table(tiles[(bright)&(small)]).write(tilefile, overwrite=True)
else:
    Table(tiles[(~bright) & (small)]).write(tilefile, overwrite=True)

print("Wrote tiles to {}".format(tilefile))

In [ ]:
# target selection results
print('Started reading {}'.format(targetfile))
targetdata = fitsio.read(targetfile, 'TARGETS')
ii = (targetdata['RA']>30) &  (targetdata['RA']<39) & (targetdata['DEC']<5) & (targetdata['DEC']>-5)
targetdata = targetdata[ii]
print('Done reading target data')

In [ ]:
plt.scatter(targetdata['RA'], targetdata['DEC'],s=0.1, alpha=0.1)
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')

In [ ]:
# write down the targets to disk

if (not os.path.exists(targetcutfile)):
    Table(targetdata).write(targetcutfile, overwrite=True)
    print('Done writing target cut data')

In [ ]:
# compute sky file
if (not os.path.exists(skycutfile)):
    skydata = fitsio.read(skyfile)
    ii = (skydata['RA']>30) &  (skydata['RA']<39) & (skydata['DEC']<5) & (skydata['DEC']>-5)
    jj = (skydata['DESI_TARGET'] & desi_mask.SKY)!=0
    skydata = skydata[ii&jj]
    Table(skydata).write(skycutfile, overwrite=True)
    print('Done writing sky cut data')
skydata = Table.read(skycutfile)

In [ ]:
plt.scatter(skydata['RA'], skydata['DEC'],s=0.1, alpha=0.1)
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')

In [ ]:
#compute MTL
if not os.path.exists(mtlfile):
    print('computing mtl')
    import desitarget.mtl
    mtl = desitarget.mtl.make_mtl(targetdata, 'DARK|GRAY')

    mtl.meta['EXTNAME'] = 'MTL'
    mtl.write(mtlfile)
    
    #print some stats
    print('MWS_TARGETS: {}'.format(np.count_nonzero(mtl['MWS_TARGET']!=0)))
    print('BGS_TARGETS: {}'.format(np.count_nonzero(mtl['BGS_TARGET']!=0)))
    print('DESI_TARGETS: {}'.format(np.count_nonzero(mtl['DESI_TARGET']!=0)))
    print('finished computing mtl')
mtl = Table.read(mtlfile)

In [ ]:
plt.scatter(mtl['RA'], mtl['DEC'], s=0.1, alpha=0.1)
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')

By default, fiberassign uses the real focalplane geometry and properties at the current time.  During operations, if a fiber broke yesterday and you run fiberassign today with an up-to-date desimodel data checkout, then that fiber will not be assigned.  For this tutorial, we will run fiberassign with a *rundate* set to a time in the past before the start of commissioning.  This will give us a nominal focalplane layout with all positioners working.

In [ ]:
assign_date = "2022-01-01T00:00:00+00:00"

In [ ]:
cmd = 'fiberassign --overwrite --mtl mtl.fits --sky sky.fits'
cmd += ' --rundate {}'.format(assign_date)
cmd += ' --footprint {}'.format(tilefile)
cmd += ' --outdir {}'.format(outdir)

print(cmd)

In [ ]:
print('RUNNING: '+cmd)
try:
    results = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT)
    print(results.decode())
    print('--- SUCCESS ---')
except subprocess.CalledProcessError as ex:
    print('--- ERROR {} ---'.format(ex.returncode))
    print(ex.output.decode())

In [ ]:
tiles = Table.read("tiles.fits")
targets = Table.read("mtl.fits")

In [ ]:
# Gather all outputs
assignments = list()
for tileid in tiles['TILEID']:
    outtile = '{}/fiberassign-{:06d}.fits'.format(outdir, tileid)
    assignments.append(Table.read(outtile, hdu=1))
    
assigned_targetids = np.concatenate([tmp['TARGETID'] for tmp in assignments])
isAssigned = np.in1d(targets['TARGETID'], assigned_targetids)

plt.figure(figsize=(12,8))
plt.plot(targets['RA'][isAssigned], targets['DEC'][isAssigned], 'k,')
plt.title('Targets assigned to fibers')
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(targets['RA'][~isAssigned], targets['DEC'][~isAssigned], 'k,')
plt.title('Targets left unassigned to fibers')
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')

We can now run some basic QA on these outputs.  We run this on the merged outputs so that we have access to more target information.

In [ ]:
cmd = 'fba_run_qa --prefix fiberassign-'
cmd += ' --rundate {}'.format(assign_date)
cmd += ' --footprint {}'.format(tilefile)
cmd += ' --dir {}'.format(outdir)

print(cmd)

In [ ]:
print('RUNNING: '+cmd)
try:
    results = subprocess.check_output(cmd.split(), stderr=subprocess.STDOUT)
    print(results.decode())
    print('--- SUCCESS ---')
except subprocess.CalledProcessError as ex:
    print('--- ERROR {} ---'.format(ex.returncode))
    print(ex.output.decode())

In [ ]:
# Load the results back in and print
import json
from pprint import PrettyPrinter

qa_file = os.path.join(outdir, "qa.json")
qa_data = None
with open(qa_file, "r") as f:
    qa_data = json.load(f)

pp = PrettyPrinter(indent=2)
pp.pprint(qa_data)